In [111]:
import os
import pandas as pd
import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Bidirectional, Dense, Embedding
from matplotlib import pyplot as plt
from tensorflow.keras.metrics import Precision
from sklearn.model_selection import train_test_split

In [112]:
df = pd.read_csv('depression_dataset_reddit_cleaned.csv')

In [113]:
df.shape

(7731, 2)

In [114]:
X = df['clean_text']
y = df['is_depression']

In [115]:
MAX_FEATURES = 250000

In [116]:
vectorizer = TextVectorization(max_tokens=MAX_FEATURES,
                               output_sequence_length=1800,
                               output_mode='int')

In [117]:
MAXLEN = 250
BATCH_SIZE = 64

data = keras.preprocessing.sequence.pad_sequences(data_sequences, maxlen=MAXLEN)

In [118]:
vectorizer.adapt(X.values)
vectorized_text = vectorizer(X.values)

In [119]:
dataset = tf.data.Dataset.from_tensor_slices((vectorized_text, y))
dataset = dataset.cache()
dataset = dataset.shuffle(7731)
dataset = dataset.batch(16)
dataset = dataset.prefetch(8)

In [120]:
train = dataset.take(int(len(dataset)*.7))
val = dataset.skip(int(len(dataset)*.7)).take(int(len(dataset)*.2))
test = dataset.skip(int(len(dataset)*.9)).take(int(len(dataset)*.1))

In [121]:
model = Sequential()
model.add(Embedding(MAX_FEATURES+1, 32))
model.add(Bidirectional(LSTM(32, activation='tanh')))
model.add(Dense(128, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [122]:
model.compile(loss='BinaryCrossentropy', optimizer='Adam', metrics=['accuracy'])

In [123]:
history = model.fit(train, epochs=1, validation_data=val)

338/338 [==============================] - 503s 1s/step - loss: 0.2648 - accuracy: 0.8937 - val_loss: 0.0776 - val_accuracy: 0.9766


In [124]:
model.evaluate(test)

48/48 [==============================] - 12s 249ms/step - loss: 0.1063 - accuracy: 0.9701


[0.10632096976041794, 0.9700520634651184]

In [125]:
from keras.models import load_model

In [126]:
model.save(os.path.join('models', 'Depression_text_classification.h5'))

In [146]:
text = vectorizer('i`m really thankfful about what i have in my life')

In [139]:
text.numpy()[:50]

array([ 110,  181,  469,    9,  604,   83,   56,  252,  369,   45,   35,
        178,    3,  821,    6,  328,   57,   66,  438, 2566,  362,   29,
          6,  355,   64,   13,   11,   15,   42,   16,   21,   84,   83,
       3428,   56,  152,  181,   16,  733,    4,   26, 2083,   65,    9,
         33,    0,    0,    0,    0,    0], dtype=int64)

In [148]:
res = model.predict(np.expand_dims(text,0))

1/1 [==============================] - 0s 135ms/step


In [150]:
if res > 0.5 :
    print("The predicted class is : Depressed") 
else:
    print("The predicted class is : Not depressed")

The predicted class is : Not depressed
